# Soft Speech Units for Improved Voice Conversion

Demo for the paper: [A Comparison of Discrete and Soft Speech Units for Improved Voice Conversion](https://ieeexplore.ieee.org/abstract/document/9746484).

- [Companion webpage](https://bshall.github.io/soft-vc/)
- [Home repo](https://github.com/bshall/soft-vc)
- [HuBERT content encoders](https://github.com/bshall/hubert)
- [Acoustic Models](https://github.com/bshall/acoustic-model)
- [HiFiGAN vocoder](https://github.com/bshall/hifigan)

In [ ]:
import torch, torchaudio
import requests
import IPython.display as display

Download the HuBERT content encoder (either hubert_soft or hubert_discrete):

In [ ]:
hubert = torch.hub.load("bshall/hubert:main", "hubert_soft").cuda()

Download the acoustic model (either hubert_soft or hubert_discrete)

In [ ]:
acoustic = torch.hub.load("bshall/acoustic-model:main", "hubert_soft").cuda()

Download the vocoder (either hifigan_hubert_soft or hifigan_hubert_discrete)

In [ ]:
hifigan = torch.hub.load("bshall/hifigan:main", "hifigan_hubert_soft").cuda()

Download an example utterance:

In [ ]:
with open("example.wav", "wb") as file:
  response = requests.get("https://drive.google.com/uc?export=preview&id=1Y3KuPAhB5VcsmIaokBVKu3LUEZOfhSu8")
  file.write(response.content)

Or upload your own:

In [ ]:
from google.colab import files

uploaded = files.upload()

Load the source audio (and resample to 16kHz if necessary)

In [ ]:
source, sr = torchaudio.load("example.wav")
source = torchaudio.functional.resample(source, sr, 16000)
source = source.unsqueeze(0).cuda()

Convert to the target speaker:

In [ ]:
with torch.inference_mode():
    # Extract speech units
    units = hubert.units(source)
    # Generate target spectrogram
    mel = acoustic.generate(units).transpose(1, 2)
    # Generate audio waveform
    target = hifigan(mel)

Lets listen to the results!

The source:

In [ ]:
display.Audio(source.squeeze().cpu(), rate=16000)

and the converted utterance:

In [ ]:
display.Audio(target.squeeze().cpu(), rate=16000)